In [1]:
library(lme4)
library(afex)
library(emmeans)
library(dplyr)
library(ggplot2)
library(svglite)
library(lmtest)
library(tidyr)
library(MCMCglmm)
library(reshape)
library(gridExtra)
library(cowplot)
library(plotrix)
library(ggsignif)

Loading required package: Matrix



************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'S', 'KR', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- Get and set global package options with: afex_options()
- Set sum-to-zero contrasts globally: set_sum_contrasts()
- For example analyses see: browseVignettes("afex")
************


Attaching package: ‘afex’


The following object is masked from ‘package:lme4’:

    lmer



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:


In [3]:
ITI <- read.csv('/home/bagotlab/eshaan.i/PRL/FP_PRL_Nov_2021/Code/Longform Data/PRLInformationMeasures_ZScore_ITI.csv')

ITI = subset(ITI, (lp > 0) & (ID!=223))
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<-ITI[,c('index','time_idx','time','ID','rewards','lp','Day','rt','PFC','vHIP','MutualInfo','CE_PFCvHIP','CE_vHIPPFC','MutualInfo_PC','CE_PFCvHIP_PC','CE_vHIPPFC_PC','MutualInfo_EI','CE_PFCvHIP_EI','CE_vHIPPFC_EI')]
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.Choice = lead(lp,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.RT = lead(rt,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Choice = lag(lp,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Reward = lag(rewards,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.2Reward = lag(Previous.Reward,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.3Reward = lag(Previous.2Reward,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.4Reward = lag(Previous.3Reward,n=240))

enditi<- as.data.frame(ITI[((ITI$time)>=8)&((ITI$time)<=10),] %>% 
                group_by(index)%>%summarize(PostChoiceMutualInfo=mean(MutualInfo_PC),PostChoiceCE_PFCvHIP=mean(CE_PFCvHIP_PC),PostChoiceCE_vHIPPFC=mean(CE_vHIPPFC_PC), EndITIMutualInfo=mean(MutualInfo_EI),EndITICE_PFCvHIP=mean(CE_PFCvHIP_EI),EndITICE_vHIPPFC=mean(CE_vHIPPFC_EI),rewards=mean(rewards),ID=mean(ID), Day=mean(Day),
                                            Previous.Reward=mean(Previous.Reward), Previous.2Reward=mean(Previous.2Reward), Previous.3Reward=mean(Previous.3Reward), Previous.4Reward=mean(Previous.4Reward),
                                            PFC=mean(PFC),vHIP=mean(vHIP)))

enditi$ID<-as.factor(enditi$ID)
enditi$Day<-as.factor(enditi$Day)
enditi$rewards <-as.factor(enditi$rewards)
enditi$Previous.Reward <-as.factor(enditi$Previous.Reward)
enditi$Previous.2Reward <-as.factor(enditi$Previous.2Reward)
enditi$Previous.3Reward <-as.factor(enditi$Previous.3Reward)
enditi$Previous.4Reward <-as.factor(enditi$Previous.4Reward)

ITI$rewards<-as.factor(ITI$rewards)
ITI$ID<-as.factor(ITI$ID)
ITI$Day<-as.factor(ITI$Day)
ITI$Previous.Reward <-as.factor(ITI$Previous.Reward)

In [6]:
#Conditional entropy of mPFC-NAc given vHip-NAc

CEvHIPPFC_rewardshist_enditi <- lmer(EndITICE_vHIPPFC~rewards*Previous.Reward+(1|ID)+(1|Day), data = enditi)
CEvHIPPFC_rewardshist_enditi_means <- emmeans(CEvHIPPFC_rewardshist_enditi, ~ rewards|Previous.Reward)
summary(CEvHIPPFC_rewardshist_enditi)
test(pairs(CEvHIPPFC_rewardshist_enditi_means, simple = "each"), by = NULL)

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 29198' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 29198)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 29198' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 29198)' or larger];
but be warned that this may result in large computation time and memory use.



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: EndITICE_vHIPPFC ~ rewards * Previous.Reward + (1 | ID) + (1 |  
    Day)
   Data: enditi

REML criterion at convergence: 18201

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4075 -0.4931  0.1061  0.6463  3.5507 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.09089  0.3015  
 Day      (Intercept) 0.01021  0.1010  
 Residual             0.10844  0.3293  
Number of obs: 29198, groups:  ID, 21; Day, 6

Fixed effects:
                            Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                1.186e+00  7.773e-02  2.402e+01  15.261 7.32e-14 ***
rewards1                  -1.524e-02  5.322e-03  2.917e+04  -2.864  0.00418 ** 
Previous.Reward1          -2.178e-03  5.321e-03  2.917e+04  -0.409  0.68232    
rewards1:Previous.Reward1 -4.843e-03  7.762e-03  2.917e+04  -0.624  0.53271    
---
Signif. codes:  0 ‘***’ 0.001 ‘*

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons



,contrast,Previous.Reward,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,rewards0 - rewards1,0,0.01524383,0.005321765,Inf,2.864432,0.0083377118
2,rewards0 - rewards1,1,0.02008651,0.005708792,Inf,3.518521,0.0008677298
,contrast,rewards,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Previous.Reward0 - Previous.Reward1,0,0.002177897,0.005321007,Inf,0.4093017,0.8990783
2,Previous.Reward0 - Previous.Reward1,1,0.007020571,0.005709344,Inf,1.2296632,0.3897629


In [7]:
#Conditional entropy of vHip-NAc given mPFC-NAc

CEPFCvHIP_rewardshist_enditi <- lmer(EndITICE_PFCvHIP~rewards*Previous.Reward+(1|ID)+(1|Day), data = enditi)
CEPFCvHIP_rewardshist_enditi_means <- emmeans(CEPFCvHIP_rewardshist_enditi, ~ rewards|Previous.Reward)
summary(CEPFCvHIP_rewardshist_enditi)
test(pairs(CEPFCvHIP_rewardshist_enditi_means, simple = "each"), by = NULL)


Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 29198' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 29198)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 29198' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 29198)' or larger];
but be warned that this may result in large computation time and memory use.



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: EndITICE_PFCvHIP ~ rewards * Previous.Reward + (1 | ID) + (1 |  
    Day)
   Data: enditi

REML criterion at convergence: 25314.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5503 -0.4765  0.0863  0.6225  4.6819 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.064268 0.25351 
 Day      (Intercept) 0.005838 0.07641 
 Residual             0.138440 0.37208 
Number of obs: 29198, groups:  ID, 21; Day, 6

Fixed effects:
                            Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                1.140e+00  6.364e-02  2.482e+01  17.915 1.04e-15 ***
rewards1                   2.320e-02  6.013e-03  2.917e+04   3.859 0.000114 ***
Previous.Reward1           8.661e-03  6.012e-03  2.917e+04   1.441 0.149676    
rewards1:Previous.Reward1 -1.342e-02  8.770e-03  2.917e+04  -1.530 0.125906    
---
Signif. codes:  0 ‘***’ 0.001 

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons



,contrast,Previous.Reward,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,rewards0 - rewards1,0,-0.023200640,0.006012792,Inf,-3.858547,0.0002281131
2,rewards0 - rewards1,1,-0.009778014,0.006450115,Inf,-1.515944,0.2422880261
,contrast,rewards,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Previous.Reward0 - Previous.Reward1,0,-0.008661483,0.006011934,Inf,-1.4407149,0.2769308
2,Previous.Reward0 - Previous.Reward1,1,0.004761143,0.006450738,Inf,0.7380773,0.7089047


In [8]:
#mPFC-NAc outcome encoding given outcome history

PFC_enditi <- lmer(PFC~Previous.3Reward*Previous.2Reward*Previous.Reward*rewards + (1|ID) + (1|Day), data = enditi)
PFC_rewardshist_enditi_means <- emmeans(PFC_enditi, ~ Previous.3Reward*Previous.2Reward*Previous.Reward*rewards)
summary(PFC_enditi)
test(pairs(PFC_rewardshist_enditi_means, simple = "each"), by = NULL)

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 28952' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 28952)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 28952' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 28952)' or larger];
but be warned that this may result in large computation time and memory use.


Correlation matrix not shown by default, as p = 16 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: PFC ~ Previous.3Reward * Previous.2Reward * Previous.Reward *  
    rewards + (1 | ID) + (1 | Day)
   Data: enditi

REML criterion at convergence: 52193.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.0608 -0.5863 -0.0661  0.5370  8.0376 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.003583 0.05986 
 Day      (Intercept) 0.001155 0.03399 
 Residual             0.353240 0.59434 
Number of obs: 28952, groups:  ID, 21; Day, 6

Fixed effects:
                                                                Estimate
(Intercept)                                                    1.027e-01
Previous.3Reward1                                             -4.443e-02
Previous.2Reward1                                             -5.130e-02
Previous.Reward1                                              -9.431e-02
rewards1                                    

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons



,contrast,Previous.2Reward,Previous.Reward,rewards,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Previous.3Reward0 - Previous.3Reward1,0,0,0,0.04442752,0.01782881,Inf,2.4918949,0.09724332
2,Previous.3Reward0 - Previous.3Reward1,1,0,0,0.02401604,0.01980921,Inf,1.2123675,0.87035772
3,Previous.3Reward0 - Previous.3Reward1,0,1,0,0.02083607,0.01868896,Inf,1.1148864,0.91473407
4,Previous.3Reward0 - Previous.3Reward1,1,1,0,-0.01112342,0.02120332,Inf,-0.5246077,0.99934275
5,Previous.3Reward0 - Previous.3Reward1,0,0,1,0.01336626,0.01963094,Inf,0.6808770,0.99583328
6,Previous.3Reward0 - Previous.3Reward1,1,0,1,0.02490758,0.02012024,Inf,1.2379367,0.85688585
7,Previous.3Reward0 - Previous.3Reward1,0,1,1,0.01564716,0.02113135,Inf,0.7404716,0.99266350
8,Previous.3Reward0 - Previous.3Reward1,1,1,1,-0.03562304,0.02166570,Inf,-1.6442135,0.57003807
,contrast,Previous.3Reward,Previous.Reward,rewards,estimate,SE,df,z.ratio,p.value


In [9]:
#vHip-NAc outcome encoding given outcome history

vHIP_enditi <- lmer(vHIP~Previous.3Reward*Previous.2Reward*Previous.Reward*rewards + (1|ID) + (1|Day), data = enditi)
vHIP_rewardshist_enditi_means <- emmeans(vHIP_enditi, ~ Previous.3Reward*Previous.2Reward*Previous.Reward*rewards)
summary(vHIP_enditi)
test(pairs(vHIP_rewardshist_enditi_means, simple = "each"), by = NULL)

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 30403' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 30403)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 30403' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 30403)' or larger];
but be warned that this may result in large computation time and memory use.


Correlation matrix not shown by default, as p = 16 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: vHIP ~ Previous.3Reward * Previous.2Reward * Previous.Reward *  
    rewards + (1 | ID) + (1 | Day)
   Data: enditi

REML criterion at convergence: 54860.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6125 -0.5808 -0.0682  0.5201  7.7550 

Random effects:
 Groups   Name        Variance  Std.Dev.
 ID       (Intercept) 0.0016540 0.04067 
 Day      (Intercept) 0.0001852 0.01361 
 Residual             0.3541496 0.59510 
Number of obs: 30403, groups:  ID, 22; Day, 6

Fixed effects:
                                                                Estimate
(Intercept)                                                    9.402e-02
Previous.3Reward1                                             -6.987e-02
Previous.2Reward1                                             -7.613e-02
Previous.Reward1                                              -1.087e-01
rewards1                               

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons

Note: adjust = "tukey" was changed to "sidak"
because "tukey" is only appropriate for one set of pairwise comparisons



,contrast,Previous.2Reward,Previous.Reward,rewards,estimate,SE,df,z.ratio,p.value
,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Previous.3Reward0 - Previous.3Reward1,0,0,0,0.06986696,0.01749951,Inf,3.9925082,0.0005229049
2,Previous.3Reward0 - Previous.3Reward1,1,0,0,0.04076083,0.01934246,Inf,2.1073236,0.2485568729
3,Previous.3Reward0 - Previous.3Reward1,0,1,0,0.02230754,0.01822517,Inf,1.2239965,0.8643228229
4,Previous.3Reward0 - Previous.3Reward1,1,1,0,-0.01235446,0.02067915,Inf,-0.5974359,0.9983249448
5,Previous.3Reward0 - Previous.3Reward1,0,0,1,0.02707252,0.01916956,Inf,1.4122660,0.7470549204
6,Previous.3Reward0 - Previous.3Reward1,1,0,1,0.04939114,0.01959459,Inf,2.5206512,0.0899570665
7,Previous.3Reward0 - Previous.3Reward1,0,1,1,0.01912063,0.02060946,Inf,0.9277597,0.9694947266
8,Previous.3Reward0 - Previous.3Reward1,1,1,1,-0.02321063,0.02117806,Inf,-1.0959755,0.9220443458
,contrast,Previous.3Reward,Previous.Reward,rewards,estimate,SE,df,z.ratio,p.value
